# Hybrid Recommendation using
a) Collaborative Filtering Model

b) Content-based Model

In [37]:
import pandas as pd
from random import randint

def generate_data(n_books = 3000, n_genres = 10, n_authors = 450, n_publishers = 50, n_readers = 30000, dataset_size = 100000):
    
    d = pd.DataFrame(
        {
            'book_id' : [randint(1, n_books) for _ in range(dataset_size)],
            'author_id' : [randint(1, n_authors) for _ in range(dataset_size)],
            'book_genre' : [randint(1, n_genres) for _ in range(dataset_size)],
            'reader_id' : [randint(1, n_readers) for _ in range(dataset_size)],
            'num_pages' : [randint(75, 700) for _ in range(dataset_size)],
            'book_rating' : [randint(1, 10) for _ in range(dataset_size)],
            'publisher_id' : [randint(1, n_publishers) for _ in range(dataset_size)],
            'publish_year' : [randint(2000, 2021) for _ in range(dataset_size)],
            'book_price' : [randint(1, 200) for _ in range(dataset_size)],
            'text_lang' : [randint(1,7) for _ in range(dataset_size)]
        }
    ).drop_duplicates()
    return d

In [38]:
df = generate_data(dataset_size = 100000)
df.to_csv('data.csv', index = False)
df.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
0,337,136,7,19155,571,10,25,2020,55,6
1,2161,74,8,28230,165,1,5,2009,87,1
2,765,382,7,9973,102,10,30,2021,48,2
3,1695,394,8,2802,565,3,43,2009,43,2
4,129,418,10,12490,479,7,36,2021,197,4


In [39]:
df = df.sort_values(by=['book_id'], ascending = True)

In [40]:
df.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
55350,1,179,2,24088,81,2,16,2013,16,7
59591,1,277,1,3840,679,1,31,2000,91,1
77078,1,25,10,11245,347,4,26,2008,169,7
29746,1,447,2,2225,340,10,35,2017,188,5
95919,1,246,5,29413,512,9,46,2011,57,7


In [41]:
import numpy as np
'''
FORMULA
X(NORMALIZED) = (X - Xminimum)/(Xmaximum - Xminimum)

 min_val = min(data)
    if min_val < 0:
        data = [x + abs(min_val) for x in data]
    max_val = max(data)
    
    return [x/max_val for x in data]
'''
#normalizing the input between 0s and 1s
def normalize(data):    
    return (data - np.min(data)/ (np.max(data)- np.min(data)))


In [42]:
''' 
CREATING FUNCTIONS FOR CONTENT BASED RECOMMENDATION MODELS

''' 

#performing one hot encoding
def encoding(df, column):
    new = pd.get_dummies(df[column])
    new.reset_index(drop = True, inplace = True)
    return pd.concat([df, new], axis = 1)

In [43]:
'''
CREATING FRUNCTION FOR COLLABORATIVE FILTER MODELLING
'''
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

def svd(mat, df, factors):
    if not 1<= factors < min(mat.shape):
        raise ValueError("Must be 1 <= factors < min(mat.shape)")
    
    #matrix factorization
    u,s,v = svds(mat, k = factors)
    s = np.diag(s)

    #calculating the prediction ratings
    pred = np.dot(np.dot(u,s),v)
    pred = normalize(pred) #normalizing the predictions

    new_df = pd.DataFrame(pred, columns = df.columns, index = list(df.index) ).transpose()

    return new_df



In [44]:
''' 
Creating a Content based recommendation syatem as the base system

class Recommender():
    def __init__(self,df):
        self.df = df
    
    def similarity(self, v1, v2):
        return np.sum(np.dot(v1,v2)/ np.cross(v1,v2)) 
    
    def recommend(self, book_id, rec):
        ip = self.df.loc[book_id].values
        self.df['sim'] = self.df.apply(lambda x: self.similarity(ip, x.values), axis =1)
        
        return self.df.nlargest(columns = 'sim', n = rec)
        
'''

" \nCreating a Content based recommendation syatem as the base system\n\nclass Recommender():\n    def __init__(self,df):\n        self.df = df\n    \n    def similarity(self, v1, v2):\n        return np.sum(np.dot(v1,v2)/ np.cross(v1,v2)) \n    \n    def recommend(self, book_id, rec):\n        ip = self.df.loc[book_id].values\n        self.df['sim'] = self.df.apply(lambda x: self.similarity(ip, x.values), axis =1)\n        \n        return self.df.nlargest(columns = 'sim', n = rec)\n        \n"

In [45]:
''' 
df['num_pages_norm'] = normalize(df['num_pages'].values)
df['book_rating_norm'] = normalize(df['book_rating'].values)
df['book_price_norm'] = normalize(df['book_price'].values)


df = encoding(df = df, column = 'publish_year')
df = encoding(df = df, column = 'book_genre')
df = encoding(df = df, column = 'text_lang')

#dropping redndant columns
cols = ['publish_year', 'book_genre', 'num_pages', 'book_rating', 'book_price', 'text_lang']
df.drop(columns = cols, inplace = True)

''' 

" \ndf['num_pages_norm'] = normalize(df['num_pages'].values)\ndf['book_rating_norm'] = normalize(df['book_rating'].values)\ndf['book_price_norm'] = normalize(df['book_price'].values)\n\n\ndf = encoding(df = df, column = 'publish_year')\ndf = encoding(df = df, column = 'book_genre')\ndf = encoding(df = df, column = 'text_lang')\n\n#dropping redndant columns\ncols = ['publish_year', 'book_genre', 'num_pages', 'book_rating', 'book_price', 'text_lang']\ndf.drop(columns = cols, inplace = True)\n\n"

In [ ]:
''' 
CREATING THE FINAL HYBRID MODEL

''' 
def hybrid(reader_id, book_id, records, data, cosine, svd):
    